# MLFLOW

MLflow experiment setup

- Automatic logging
- Training multiple models (LGBM, XGB, LR, DT, etc.)
- Saving metrics, params & artifacts
- Registering the best model

In [1]:
import mlflow
import mlflow.sklearn
import mlflow.lightgbm
import mlflow.xgboost

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score


In [2]:
train_df = pd.read_csv("../data/processed/train.csv")
val_df   = pd.read_csv("../data/processed/val.csv")
test_df  = pd.read_csv("../data/processed/test.csv")

X_train, y_train = train_df.drop("Default", axis=1), train_df["Default"]
X_val, y_val     = val_df.drop("Default", axis=1), val_df["Default"]
X_test, y_test   = test_df.drop("Default", axis=1), test_df["Default"]


In [3]:
def evaluate_model(model, X, y):
    preds = model.predict_proba(X)[:,1]
    preds_label = (preds > 0.5).astype(int)
    
    return {
        "accuracy": accuracy_score(y, preds_label),
        "auc": roc_auc_score(y, preds),
        "f1": f1_score(y, preds_label),
        "recall": recall_score(y, preds_label)
    }


In [4]:
mlflow.set_experiment("loan_default_predictions")

e:\Loan-Defaulter\ml-venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/26 18:58:31 INFO mlflow.tracking.fluent: Experiment with name 'loan_default_predictions' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///e:/Loan-Defaulter/notebooks/mlruns/780895840933422982', creation_time=1764163711150, experiment_id='780895840933422982', last_update_time=1764163711150, lifecycle_stage='active', name='loan_default_predictions', tags={}>

In [13]:
import lightgbm as lgb

with mlflow.start_run(run_name="LightGBM_v1"):

    params = {
        "learning_rate": 0.02,
        "num_leaves": 64,
        "max_depth": -1,
        "n_estimators": 1200
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)])

    metrics = evaluate_model(model, X_val, y_val)

    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.lightgbm.log_model(model, "model")

    print("Logged LightGBM:", metrics)


[LightGBM] [Info] Number of positive: 6891, number of negative: 78408
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004764 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3523
[LightGBM] [Info] Number of data points in the train set: 85299, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.080786 -> initscore=-2.431710
[LightGBM] [Info] Start training from score -2.431710


2025/11/26 19:03:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 19:03:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged LightGBM: {'accuracy': 0.9208337892548419, 'auc': 0.7552442815309417, 'f1': 0.0670535138620245, 'recall': 0.035206499661475966}


In [6]:
import xgboost as xgb

with mlflow.start_run(run_name="XGBoost_v1"):

    params = {
        "eta": 0.1,
        "max_depth": 6,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "eval_metric": "auc"
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    metrics = evaluate_model(model, X_val, y_val)

    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.xgboost.log_model(model, "model")

    print("Logged XGB:", metrics)


2025/11/26 18:58:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 18:58:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged XGB: {'accuracy': 0.9193566035671299, 'auc': 0.746215697819515, 'f1': 0.03407601572739188, 'recall': 0.017603249830737983}


In [7]:
from sklearn.linear_model import LogisticRegression

with mlflow.start_run(run_name="LogisticRegression_v1"):

    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    metrics = evaluate_model(model, X_val, y_val)

    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model, "model")

    print("Logged Logistic Regression:", metrics)


e:\Loan-Defaulter\ml-venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/11/26 18:58:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 18:58:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged Logistic Regression: {'accuracy': 0.9191924718240507, 'auc': 0.5944563057370323, 'f1': 0.0, 'recall': 0.0}


In [8]:
from sklearn.tree import DecisionTreeClassifier

with mlflow.start_run(run_name="DecisionTree_v1"):

    params = {"max_depth": 10, "min_samples_split": 20}

    model = DecisionTreeClassifier(**params)
    model.fit(X_train, y_train)

    metrics = evaluate_model(model, X_val, y_val)

    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(model, "model")

    print("Logged Decision Tree:", metrics)


2025/11/26 18:58:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 18:58:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged Decision Tree: {'accuracy': 0.9144326512747566, 'auc': 0.6907256423181761, 'f1': 0.07674144037780402, 'recall': 0.044008124576844956}


In [14]:
mlflow.search_runs(experiment_names=["loan_default_predictions"])

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.auc,metrics.f1,metrics.recall,metrics.accuracy,...,params.learning_rate,params.min_samples_split,params.colsample_bytree,params.eval_metric,params.subsample,params.eta,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.runName,tags.mlflow.source.name
0,3d7dfe6a6dfd4c529bbfeba22b0cb39e,780895840933422982,FINISHED,file:///e:/Loan-Defaulter/notebooks/mlruns/780...,2025-11-26 13:33:14.782000+00:00,2025-11-26 13:33:23.819000+00:00,0.755244,0.067054,0.035206,0.920834,...,0.02,None,None,None,None,None,ompan,LOCAL,LightGBM_v1,e:\Loan-Defaulter\ml-venv\Lib\site-packages\ip...
1,43ed1651742545e194619dc9aeb13ce3,780895840933422982,FINISHED,file:///e:/Loan-Defaulter/notebooks/mlruns/780...,2025-11-26 13:28:48.246000+00:00,2025-11-26 13:28:52.097000+00:00,0.690726,0.076741,0.044008,0.914433,...,None,20,None,None,None,None,ompan,LOCAL,DecisionTree_v1,e:\Loan-Defaulter\ml-venv\Lib\site-packages\ip...
2,4edbfdf0c8f24eb1809f8ac6e815415b,780895840933422982,FINISHED,file:///e:/Loan-Defaulter/notebooks/mlruns/780...,2025-11-26 13:28:44.410000+00:00,2025-11-26 13:28:48.182000+00:00,0.594456,0.000000,0.000000,0.919192,...,None,None,None,None,None,None,ompan,LOCAL,LogisticRegression_v1,e:\Loan-Defaulter\ml-venv\Lib\site-packages\ip...
3,036dd0ad3ab148669f121db90d9dd692,780895840933422982,FINISHED,file:///e:/Loan-Defaulter/notebooks/mlruns/780...,2025-11-26 13:28:39.144000+00:00,2025-11-26 13:28:44.370000+00:00,0.746216,0.034076,0.017603,0.919357,...,None,None,0.8,auc,0.8,0.1,ompan,LOCAL,XGBoost_v1,e:\Loan-Defaulter\ml-venv\Lib\site-packages\ip...
4,a42bae6e509e48478a78f23cde08d3c3,780895840933422982,FINISHED,file:///e:/Loan-Defaulter/notebooks/mlruns/780...,2025-11-26 13:28:31.549000+00:00,2025-11-26 13:28:39.110000+00:00,0.747283,0.026350,0.013541,0.919138,...,0.05,None,None,None,None,None,ompan,LOCAL,LightGBM_v1,e:\Loan-Defaulter\ml-venv\Lib\site-packages\ip...


In [15]:
best_run_id = "3d7dfe6a6dfd4c529bbfeba22b0cb39e"

mlflow.register_model(
    f"../runs:/{best_run_id}/model",
    "LoanDefaultModel"
)


Registered model 'LoanDefaultModel' already exists. Creating a new version of this model...
Created version '4' of model 'LoanDefaultModel'.


<ModelVersion: aliases=[], creation_timestamp=1764164063033, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1764164063033, metrics=None, model_id=None, name='LoanDefaultModel', params=None, run_id=None, run_link=None, source='../runs:/3d7dfe6a6dfd4c529bbfeba22b0cb39e/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [16]:
model = mlflow.pyfunc.load_model("models:/LoanDefaultModel/1")
test_probs = model.predict(X_test)
test_preds = (test_probs >= 0.5).astype(int)


In [17]:
accuracy = accuracy_score(y_test, test_preds)
auc = roc_auc_score(y_test, test_probs)
f1 = f1_score(y_test, test_preds)
recall = recall_score(y_test, test_preds)

threshold_5 = np.percentile(test_probs, 95)
recall_top5 = recall_score(y_test, (test_probs >= threshold_5).astype(int))

print("Accuracy:", accuracy)
print("AUC:", auc)
print("F1:", f1)
print("Recall:", recall)
print("Recall@Top5%:", recall_top5)

Accuracy: 0.9137261338147601
AUC: 0.5183289952343907
F1: 0.080466472303207
Recall: 0.046716316858496955
Recall@Top5%: 1.0


The optimising algorithms focus on AUC-ROC. It is easier to select models on platforms like AWS, Kubeflow.